In [1]:
import os
os.chdir("drive/MyDrive/DA6401/DA6401_A3/")

In [2]:
import pandas as pd
train_file = "lexicons/ta.translit.sampled.train.tsv"
train_data = pd.read_csv (train_file, header=None, sep='\t')

In [3]:
train_data

,0,1,2
0,ஃபியட்,fiat,2
1,ஃபியட்,phiyat,1
2,ஃபியட்,piyat,1
3,ஃபிரான்ஸ்,firaans,1
4,ஃபிரான்ஸ்,france,2
...,...,...,...
68213,ஹோல்ட்,holtt,1
68214,ஹோல்ட்,hoold,1
68215,ஹோல்ட்,hoolt,1
68216,ஹோல்ட்,hooltt,1


In [4]:
import numpy as np
import keras

In [11]:
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

In [6]:
input_texts = []
target_texts = []

#Adding "_" as a padding character
input_characters = set('_')
target_characters = set('_')
for index, row in train_data.iterrows():
    input_text, target_text, attesters = row[0], row[1], row[2]
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    if isinstance(target_text, str) != True or isinstance(input_text, str) != True:
      continue
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 68215
Number of unique input tokens: 47
Number of unique output tokens: 29
Max sequence length for inputs: 26
Max sequence length for outputs: 32


In [7]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index["_"]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index["_"]] = 1.0
    decoder_target_data[i, t:, target_token_index["_"]] = 1.0

In [8]:
#currently padding is not masked

encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)

#cell state is s_t, hidden state is h_t
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [13]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s_model.keras")


Epoch 1/10
853/853 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.8888 - loss: 0.3498 - val_accuracy: 0.8133 - val_loss: 0.7300
Epoch 2/10
853/853 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9004 - loss: 0.3138 - val_accuracy: 0.8253 - val_loss: 0.6765
Epoch 3/10
853/853 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.9119 - loss: 0.2774 - val_accuracy: 0.8319 - val_loss: 0.6603
Epoch 4/10
853/853 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.9218 - loss: 0.2447 - val_accuracy: 0.8357 - val_loss: 0.6438
Epoch 5/10
853/853 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.9290 - loss: 0.2209 - val_accuracy: 0.8414 - val_loss: 0.6284
Epoch 6/10
853/853 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9343 - loss: 0.2034 - val_accuracy: 0.8466 - val_loss: 0.6441
Epoch 7/10
853/853 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9389 - loss: 0.1872 - val_accuracy: 0.8489 - val_loss: 0.6121
Epoch 8/10
853/853 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.9426 - loss: 0.1737 - val_ac

In [15]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s_model.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence


In [16]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


-
Input sentence: ஃபியட்
Decoded sentence: piyat

-
Input sentence: ஃபியட்
Decoded sentence: piyat

-
Input sentence: ஃபியட்
Decoded sentence: piyat

-
Input sentence: ஃபிரான்ஸ்
Decoded sentence: franc

-
Input sentence: ஃபிரான்ஸ்
Decoded sentence: franc

-
Input sentence: ஃபிரான்ஸ்
Decoded sentence: franc

-
Input sentence: ஃபிரான்ஸ்
Decoded sentence: franc

-
Input sentence: ஃபிலிம்
Decoded sentence: filim

-
Input sentence: ஃபிலிம்
Decoded sentence: filim

-
Input sentence: ஃபிலிம்
Decoded sentence: filim

-
Input sentence: ஃபிலிம்ஸ்
Decoded sentence: filmm

-
Input sentence: ஃபிலிம்ஸ்
Decoded sentence: filmm

-
Input sentence: ஃபிலிம்ஸ்
Decoded sentence: filmm

-
Input sentence: ஃபேஸ்புக்
Decoded sentence: basseep

-
Input sentence: ஃபேஸ்புக்
Decoded sentence: basseep

-
Input sentence: ஃபேஸ்புக்
Decoded sentence: basseep

-
Input sentence: ஃபோன்
Decoded sentence: pon

-
Input sentence: ஃபோன்
Decoded sentence: pon

-
Input sentence: ஃபோன்
Decoded sentence: pon

-
Input sentence: ஃப

In [18]:
!du -h

3.2M	./lexicons
30K	./.git/hooks
4.0K	./.git/branches
4.5K	./.git/info
4.5K	./.git/refs/heads
4.0K	./.git/refs/tags
4.5K	./.git/refs/remotes/origin
8.5K	./.git/refs/remotes
21K	./.git/refs
6.5K	./.git/objects/pack
4.0K	./.git/objects/info
15K	./.git/objects
4.5K	./.git/logs/refs/remotes/origin
8.5K	./.git/logs/refs/remotes
4.5K	./.git/logs/refs/heads
17K	./.git/logs/refs
22K	./.git/logs
102K	./.git
8.0M	.
